# Using an external solver discrete implicit modelling
The core to discrete implicit modelling is solving a series of linear equations in a least squares sense. 
The `DiscreteInterpolator` class handles the assembly of this system for the geological observations and the regularisation terms. 
The solution to the least squares problem can be found by finding the solution to the matrix equation:
\begin{equation}
Ax = b
\end{equation}
The matrix A is a $n x m$ matrix where $n$ is the number of equations (interpolation constraints + regularisation constraints) and $m$ is the number of nodes for the discrete support.  
Most solvers require a square matrix. The equations above can be transformed into a square system where the transpose of A ($A^T$) is multiplied by the matrix system of equations becoming:
\begin{equation}
A^TAx=A^Tb
\end{equation}

LoopStructural implements different solvers for finding the solution to these equations - the default solver is the iterative conjugate gradeint method from scipy. Alternatives are the cholesky decomposition provided by suitesparse and scikit-sparse as well as the lower upper decomposition provided by scipy.

This tutorial will demonstrate how an external solver can be used for solving the system of equations. 

The tutorial will use the dataset from [quickstart implicit surface modelling](1.&#32;Quickstart&#32;implicit&#32;surface&#32;modelling&#32;-&#32;Claudius&#32;data.ipynb)

### Setting up the interpolator

In [3]:
from LoopStructural.interpolators.piecewiselinear_interpolator import PiecewiseLinearInterpolator as PLI
from LoopStructural.supports.tet_mesh import TetMesh
from LoopStructural.modelling.features.geological_feature import GeologicalFeatureInterpolator
from LoopStructural.visualisation.model_visualisation import LavaVuModelViewer

import numpy as np
import lavavu
import matplotlib.pyplot as plt
import pandas as pd
import glob
%matplotlib inline

In [4]:
dips = pd.read_csv('data/Dips.csv',delimiter=';')
# import all of the csv into the same dataframe use glob to find all files matching pattern
dfs = []
for f in glob.glob('data/*Points.csv'):
    dfs.append(pd.read_csv(f,delimiter=';'))
points = pd.concat(dfs,axis=0,ignore_index=True)

dfs = []
for f in glob.glob('data/*Section.csv'):
    dfs.append(pd.read_csv(f,delimiter=';'))
sections = pd.concat(dfs,axis=0,ignore_index=True)
boundary_points = np.zeros((2,3))
boundary_points[0,0] = 548800
boundary_points[0,1] = 7816600
boundary_points[0,2] = -11010
boundary_points[1,0] = 552500
boundary_points[1,1] = 7822000
boundary_points[1,2] = -8400

# build the mesh
mesh = TetMesh()
mesh.setup_mesh(boundary_points, n_tetra=50000,)

# link mesh to the interpolator 
interpolator = PLI(mesh)
stratigraphy_builder = GeologicalFeatureInterpolator(
    interpolator=interpolator,
    name='stratigraphy')
solver = 'cg'
for i, r in points.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in sections.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in dips.iterrows():
    stratigraphy_builder.add_planar_constraint([r['X'],r['Y'],r['Z']],[r['OrientX'],r['OrientY'],r['OrientZ']])


### Define an external solver as a function
An external solver can be passed to the feature builder by using the kwarg `solver=external` and `external=solve` where solve is a function of the form below. For example, below we use pyamg 
```Python
def solve(A,B):
    return c
```

In [7]:
import pyamg
def solve_pyamg(A,B):
#     m1 = pyamg.solve(A)
    m1 = pyamg.smoothed_aggregation_solver(A)
    return m1.solve(B,maxiter=100,tol=1e-8,accel='cg')

In [8]:
stratigraphy = stratigraphy_builder.build(solver='external',external=solve_pyamg,cpw=1,cgw=0.1)

2019-10-09 17:34:16,428 LoopStructural.interpolators.discete_interpolator WARNING  Using external solver
